In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
np.random.seed(43)

In [ ]:
source_file = './WF_Rohdaten.xlsx'
current_calendar_file = './NBDF_schleppkalender_pull_2019-04-30.xlsx'
replies = './Rückmeldungne.xlsx'

In [ ]:
results_file = f'Filled_calendar_{datetime.now().strftime("%Y-%m-%d")}.xlsx'

In [ ]:
NBDF_member = pd.read_excel(source_file)
current_calendar = pd.read_excel(current_calendar_file)
replies_dict = {}
for sheet in pd.ExcelFile(replies).sheet_names:
    try:
        df = pd.read_excel(replies,sheet_name=sheet)
        first_name = df.columns[7]
        last_name = str(df[first_name][0])
        name = f'{first_name.strip().capitalize()[0]}. {last_name.strip().capitalize()}'
        unavaliable = list(df[(df['Kann ich auf keinen Fall']=='X') | (df['Kann ich auf keinen Fall']=='x') | (df['Kann ich auf keinen Fall']=='1')]['Datum'])
        replies_dict[name] = unavaliable
    except:
        pass

In [ ]:
eingeteilte_WF = list(current_calendar['Windenfahrer'].dropna())
eingeteilte_helfer = list(current_calendar['Helfer'].dropna())

In [ ]:
wf_dict = {}
for wf in eingeteilte_WF:
    if wf in wf_dict.keys():
        wf_dict[wf]+=1
    else:
        wf_dict[wf]= 1
        
helfer_dict = {}
for helfer in eingeteilte_helfer:
    if helfer in helfer_dict.keys():
        helfer_dict[helfer]+=1
    else:
        helfer_dict[helfer]=1

In [ ]:
wf_dict

In [ ]:
helfer_dict

In [ ]:
NBDF_member['Name_short'] = NBDF_member.apply(lambda row: row['Vorname'][0]+'. ' + row['Name'], axis=1)

In [ ]:
NBDF_Helfer = NBDF_member[NBDF_member['WF HG'] != 'X']

In [ ]:
Helfer_Namen = list(NBDF_Helfer['Name_short'])
Helfer_Schlepps = [int(s) for s in list(NBDF_Helfer['Anzahl Schlepps 2018'])]
total_schlepps = sum(Helfer_Schlepps)
Helfer_Prob = [s/total_schlepps for s in Helfer_Schlepps]

In [ ]:
WF_GS = NBDF_member[['Name_short','Anzahl Schlepps 2018','WF HG']]
WF_GS = WF_GS[WF_GS['WF HG'].isin(['X'])]

In [ ]:
WF_GS.sort_values(['Anzahl Schlepps 2018'],ascending=False)

In [ ]:
verfügbare_wf = sorted(list(set(list(WF_GS['Name_short'])+eingeteilte_WF)))

In [ ]:
verfügbare_wf

In [ ]:
besetzte_liste = []
max_iter = 20
for wf in verfügbare_wf:
    if not wf in replies_dict.keys():
        replies_dict[wf] = []
dates = list(current_calendar['Datum'])
for idx, wf in enumerate(list(current_calendar['Windenfahrer'])):
    if str(wf) != 'nan':
        besetzte_liste.append(wf)
    else:
        geringster_wert = min(wf_dict.values())
        haben_genug = [key for key, value in wf_dict.items() if value > geringster_wert]
        if len(haben_genug) == 0:
            haben_genug = []
        n_iter = 0
        while True:
            n_iter += 1
            kandidat = np.random.choice(verfügbare_wf) 
            if n_iter > max_iter:
                besetzte_liste.append(kandidat)
                if kandidat in wf_dict.keys():
                    wf_dict[kandidat]+=1
                else:
                    wf_dict[kandidat]=1
                break
            if not kandidat in haben_genug:
                c_date = dates[idx]
                if not c_date in replies_dict[kandidat]:
                    besetzte_liste.append(kandidat)
                    if kandidat in wf_dict.keys():
                        wf_dict[kandidat]+=1
                    else:
                        wf_dict[kandidat]=1
                    break
current_calendar['Windenfahrer voll besetzt'] = besetzte_liste

In [ ]:
besetzte_liste = []
max_iter = 30
for hf in Helfer_Namen:
    if not hf in replies_dict.keys():
        replies_dict[hf] = []
    if not hf in helfer_dict.keys():
        helfer_dict[hf] = 0
dates = list(current_calendar['Datum'])
for idx, wf in enumerate(list(current_calendar['Helfer'])):
    if str(wf) != 'nan':
        besetzte_liste.append(wf)
    else:
        geringster_wert = min(helfer_dict.values())
        haben_genug = [key for key, value in helfer_dict.items() if value > geringster_wert]
        if len(haben_genug) == 0:
            haben_genug = []
        n_iter = 0
        while True:
            n_iter += 1
            kandidat = np.random.choice(Helfer_Namen,p=Helfer_Prob)
            print(kandidat)
            if n_iter > max_iter:
                besetzte_liste.append(kandidat)
                if kandidat in wf_dict.keys():
                    helfer_dict[kandidat]+=1
                else:
                    helfer_dict[kandidat]=1
                break
            if not kandidat in haben_genug:
                c_date = dates[idx]
                if not c_date in replies_dict[kandidat]:
                    besetzte_liste.append(kandidat)
                    if kandidat in wf_dict.keys():
                        helfer_dict[kandidat]+=1
                    else:
                        helfer_dict[kandidat]=1
                    break
current_calendar['Helfer voll besetzt'] = besetzte_liste

In [ ]:
current_calendar

In [ ]:
wf_dict

In [ ]:
helfer_dict

In [ ]:
neue_helfer = current_calendar[current_calendar['Helfer'].isnull()].sort_values('Helfer voll besetzt').reset_index()[['Datum','Helfer voll besetzt']]

In [ ]:
neue_wf = current_calendar[current_calendar['Windenfahrer'].isnull()].sort_values('Windenfahrer voll besetzt').reset_index()[['Datum','Windenfahrer voll besetzt']]

In [ ]:
neue_wf

In [ ]:
with pd.ExcelWriter(results_file) as writer:
    current_calendar.to_excel(writer, sheet_name='calendar',index=False)
    neue_wf.to_excel(writer, sheet_name='neue_wf')
    neue_helfer.to_excel(writer, sheet_name='neue_helfer')
print('All Done')